In [4]:
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

import torch.optim as optim
from torch.utils.data import DataLoader

In [5]:
# pth_models = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/2801_models_Apple.csv"
pth_models = "/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_models_Apple.csv"
df_models = pd.read_csv(pth_models, sep=';')

pth_offers = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_Apple_triplets_offer_model_train.csv'
df_offers = pd.read_csv(pth_offers, sep=';')

pth_offers_test = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_test_Apple.csv'
df_offers_test = pd.read_csv(pth_offers, sep=';')

id_category = {
    # 3902: 'диктофоны, портативные рекордеры',
    # 510402: 'электронные книги',
    # 4302: 'автомобильные телевизоры, мониторы',
    # 2815: 'смарт-часы и браслеты',
    # 3901: 'портативные медиаплееры',
    # 3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    # 510401: 'планшетные компьютеры и мини-планшеты',
    # 2102: 'наушники, гарнитуры, наушники c микрофоном',
    # 3903: 'радиоприемники, радиобудильники, радиочасы',
    # 3907: 'магнитолы',
    # 280801: 'GPS-навигаторы'
    }

df_models = df_models[df_models['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers[df_offers['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers#[:10000]

df_offers_shuffled = df_offers.sample(frac=1, random_state=42)

# Определяем размер тестовой выборки (например, 20%)
test_size = int(0.15 * len(df_offers_shuffled))

# Разделяем данные на тренировочную и тестовую выборки
df_train = df_offers_shuffled.iloc[:-test_size]
df_test = df_offers_shuffled.iloc[-test_size:]

# Пример: вывод размеров тренировочной и тестовой выборок
print("Размер тренировочной выборки:", len(df_train))
print("Размер тестовой выборки:", len(df_test))


Размер тренировочной выборки: 36312
Размер тестовой выборки: 6408


In [6]:
from interface import get_query_emb_batch, load_model, cosine_similarity_batch
from typing import Tuple, List, Dict, Union, Any
from colbert.modeling.checkpoint import Checkpoint

# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# experiment = "colbert-5387"

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/2801_lr04_bsize_210_apple/none/2024-04/18/09.16.10/checkpoints/colbert-187-finish"
experiment = "colbert-187-finish"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"
checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

In [7]:
from torch.utils.data import DataLoader, TensorDataset, Dataset

def get_query_emb(sentences: List[str], checkpoint: Checkpoint, batch_size: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.

    Returns:
        np.ndarray: An array of embeddings for the input sentences.
    """
    return checkpoint.queryFromText(sentences, bsize=batch_size).to("cpu").numpy()

def get_query_emb_batch(sentences: List[str], checkpoint: Checkpoint, batch_size: int, batch_size2: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences in batches using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.
        batch_size2 (int): The size of the sub-batches to split the input sentences into.

    Returns:
        np.ndarray: An array of embeddings for the input sentences. 
        Shape of the array is (len(sentences), 32, 768) for bert-base-multilingual-cased or (len(sentences), 32, 128) for colbertv2.0
    """
    embeddings_list = []
    
    for i in range(0, len(sentences), batch_size2):
        # print(f"batch: {min(i+batch_size2, len(sentences))}/{len(sentences)}")

        batch_sentences = sentences[i:i+batch_size2]
        embeddings = get_query_emb(batch_sentences, checkpoint, batch_size)
        embeddings_list.append(embeddings)

        torch.cuda.empty_cache()
    
    combined_embeddings = np.concatenate(embeddings_list, axis=0)
    return combined_embeddings

class MyDataset_old(Dataset):
    def __init__(self, offer_batch, true_match_batch, false_match_batch, checkpoint):
        self.offer_batch = offer_batch
        self.true_match_batch = true_match_batch
        self.false_match_batch = false_match_batch
        self.checkpoint = checkpoint

    def __len__(self):
        return len(self.offer_batch)

    def __getitem__(self, idx):
        offer_embs = get_query_emb_batch([self.offer_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)
        true_match_embs = get_query_emb_batch([self.true_match_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)
        false_match_embs = get_query_emb_batch([self.false_match_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)

        y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
        y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])

        X_true = np.stack([offer_embs, true_match_embs], axis=1)
        X_false = np.stack([offer_embs, false_match_embs], axis=1)

        X = np.concatenate([X_true, X_false])
        y = np.concatenate([y_true, y_false])

        X_tensor = torch.tensor(X.reshape(X.shape[0], -1), dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32)  # изменено на float32

        # print(f"offer_embs: {np.shape(offer_embs)}")
        # print(f"true_match_embs: {np.shape(true_match_embs)}")
        # print(f"false_match_embs: {np.shape(false_match_embs)}")
        # print(f"y_true: {np.shape(y_true)}")
        # print(f"y_false: {np.shape(y_false)}")
        # print(f"X_true: {np.shape(X_true)}")
        # print(f"X_false: {np.shape(X_false)}")
        # print(f"X: {np.shape(X)}")
        # print(f"y: {np.shape(y)}")
        # print(f"X_tensor: {np.shape(X_tensor)}")
        # print(f"y_tensor: {np.shape(y_tensor)}")

        return X_tensor, y_tensor
    
class MyDataset(Dataset):
    def __init__(self, offer_batch, true_match_batch, false_match_batch, checkpoint):
        self.offer_batch = offer_batch
        self.true_match_batch = true_match_batch
        self.false_match_batch = false_match_batch
        self.checkpoint = checkpoint

    def __len__(self):
        return len(self.offer_batch)

    def __getitem__(self, idx):
        offer_embs = get_query_emb_batch([self.offer_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)
        true_match_embs = get_query_emb_batch([self.true_match_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)
        false_match_embs = get_query_emb_batch([self.false_match_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)

        y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
        y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])

        X = np.concatenate([offer_embs, offer_embs])#.reshape(-1,32,768)
        X_pair = np.concatenate([true_match_embs, false_match_embs])#.reshape(-1,32,768)
        y = np.concatenate([y_true, y_false])#.reshape(-1)

        # X_tensor = torch.tensor(X.reshape(X.shape[0], -1), dtype=torch.float32)
        # X_pair_tensor = torch.tensor(X_pair.reshape(X_pair.shape[0], -1), dtype=torch.float32)
        # y_tensor = torch.tensor(y, dtype=torch.float32)  # изменено на float32

        X_tensor = torch.tensor(X, dtype=torch.float32)
        X_pair_tensor = torch.tensor(X_pair, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32)


        # print(f"offer_embs: {np.shape(offer_embs)}")
        # print(f"true_match_embs: {np.shape(true_match_embs)}")
        # print(f"false_match_embs: {np.shape(false_match_embs)}")
        # print(f"y_true: {np.shape(y_true)}")
        # print(f"y_false: {np.shape(y_false)}")
        # print(f"X_true: {np.shape(X_true)}")
        # print(f"X_false: {np.shape(X_false)}")
        # print(f"X: {np.shape(X)}")
        # print(f"y: {np.shape(y)}")
        # print(f"X_tensor: {np.shape(X_tensor)}")
        # print(f"y_tensor: {np.shape(y_tensor)}")

        return X_tensor, X_pair_tensor, y_tensor
    
# Используем созданный класс для создания загрузчика данных
df = df_train.copy()
offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])
train_dataset = MyDataset(offer_batch, true_match_batch, false_match_batch, checkpoint)
train_dataloader = DataLoader(train_dataset, batch_size=1000, shuffle=True)

df = df_test.copy()
offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])
test_dataset = MyDataset(offer_batch, true_match_batch, false_match_batch, checkpoint)
test_dataloader = DataLoader(train_dataset, batch_size=1000, shuffle=True)

# for X_tensor, X_pair_tensor, y_tensor in train_dataloader:
#     print(f"X_tensor: {np.shape(X_tensor)}")
#     print(f"X_pair_tensor: {np.shape(X_pair_tensor)}")
#     print(f"y_tensor: {np.shape(y_tensor)}")
#     break

In [8]:
# X_tensor: torch.Size([100, 2, 24576])
# X_pair_tensor: torch.Size([100, 2, 24576])
# y_tensor: torch.Size([100, 2])

In [9]:
# X_tensor: torch.Size([100, 2, 32, 768])
# X_pair_tensor: torch.Size([100, 2, 32, 768])
# y_tensor: torch.Size([100, 2])

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_dim=(32, 768), latent_dim=128):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim[0] * input_dim[1], 1024)  # Flatten input
        self.fc2 = nn.Linear(1024, 256)
        self.fc31 = nn.Linear(256, latent_dim)
        self.fc32 = nn.Linear(256, latent_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the matrix to a vector
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)

class Decoder(nn.Module):
    def __init__(self, latent_dim=128, output_dim=(32, 768)):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 256)
        self.fc2 = nn.Linear(256, 1024)
        self.fc3 = nn.Linear(1024, output_dim[0] * output_dim[1])
        self.output_dim = output_dim

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return torch.sigmoid(self.fc3(h)).view(-1, self.output_dim[0], self.output_dim[1])  # Reshape to original

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        return self.decoder(z), mu, log_var

class SiameseNetwork(nn.Module):
    def __init__(self, latent_dim=128):
        super(SiameseNetwork, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, z1, z2):
        h1 = F.relu(self.fc1(z1))
        h2 = F.relu(self.fc1(z2))
        diff = torch.abs(h1 - h2)
        out = F.relu(self.fc2(diff))
        return torch.sigmoid(self.fc3(out)).squeeze(1)  # Ensure this line produces a shape of [batch_size]


In [11]:
class IgorDataset():

  def __init__(self, train=False):
      self.means = [-47, -19,  69,  15,  87,  46, -72, -32, -89,  82]
      self.stds = [1.23, 0.05, 0.68, 0.29, 0.09, 2.19, 0.27, 1.52, 1.27, 0.66]
      self.sims_threshold_mean = 5
      self.sims_threshold_std = 0.05

      self.balance_probability = 0.5

      self.input_shape = (32, 768)

      self.global_min = min(self.means) - self.sims_threshold_mean - 10
      self.global_max = max(self.means) + self.sims_threshold_mean + 10

  def __len__(self):
      return 100

  def __getitem__(self, idx):
      f_mean = self.means[idx % len(self.means)]
      f_std = self.stds[idx % len(self.stds)]

      similarity = 0
      if np.random.choice([True, False], p=[self.balance_probability, 1-self.balance_probability]):
        g_mean = f_mean + np.random.randint(-self.sims_threshold_mean, self.sims_threshold_mean)
        g_std = f_std + np.random.uniform(-self.sims_threshold_std, self.sims_threshold_std)
        similarity = 1
      else:
        rand_index = np.random.randint(len(self.means))
        while rand_index == idx % len(self.means):
            rand_index = np.random.randint(len(self.means))

        g_mean = self.means[rand_index]
        g_std = self.stds[rand_index]
        g_mean = g_mean + np.random.randint(-self.sims_threshold_mean, self.sims_threshold_mean)
        g_std = g_std + np.random.uniform(-self.sims_threshold_std, self.sims_threshold_std)

      f = np.random.normal(f_mean, f_std, self.input_shape)
      g = np.random.normal(g_mean, g_std, self.input_shape)

      f = (f - self.global_min) / (self.global_max - self.global_min)
      g = (g - self.global_min) / (self.global_max - self.global_min)

      # return (f, g), similarity
      return torch.tensor(f, dtype=torch.float32), torch.tensor(g, dtype=torch.float32), torch.tensor([float(similarity)], dtype=torch.float32)
  

In [12]:
def vae_loss(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

def validate(vae, siamese, dataloader):
    vae.eval()
    siamese.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    mean_vae_time = 0
    mean_seamese_time = 0

    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for f, g, similarity in dataloader:
            # Process through VAE
            f, g, similarity = f.reshape(-1,32,768), g.reshape(-1,32,768), similarity.reshape(-1)

            recon_f, mu_f, log_var_f = vae(f.view(f.size(0), -1))
            recon_g, mu_g, log_var_g = vae(g.view(g.size(0), -1))

            vae_time_start = time.time()
            loss_vae_f = vae_loss(recon_f, f, mu_f, log_var_f)
            loss_vae_g = vae_loss(recon_g, g, mu_g, log_var_g)
            mean_vae_time += time.time() - vae_time_start

            siamese_time_start = time.time()
            similarity_score = siamese(vae.reparameterize(mu_f, log_var_f), vae.reparameterize(mu_g, log_var_g))
            mean_seamese_time += time.time() - siamese_time_start

            loss_siamese = F.binary_cross_entropy(similarity_score, similarity.view(-1))

            loss = loss_vae_f + loss_vae_g + loss_siamese

            # Update total validation loss
            total_loss += loss.item()

            # Accuracy calculation
            predicted_labels = (similarity_score > 0.5).float()
            correct_predictions += (predicted_labels == similarity.view(-1)).sum().item()
            total_samples += similarity.size(0)

            # Collect all predictions and actual labels for ROC AUC calculation
            all_predictions.extend(similarity_score.detach().cpu().numpy())
            all_targets.extend(similarity.cpu().numpy())


    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples
    roc_auc = roc_auc_score(all_targets, all_predictions)  # Compute ROC AUC
    mean_vae_time = mean_vae_time / total_samples
    mean_seamese_time = mean_seamese_time / total_samples

    return average_loss, accuracy, roc_auc, mean_vae_time, mean_seamese_time

def train(vae, siamese, train_dataloader, val_dataloader, optimizer, epochs=15):
    for epoch in range(epochs):
        vae.train()
        siamese.train()
        total_train_loss = 0
        for f, g, similarity in train_dataloader:
            f, g, similarity = f.reshape(-1,32,768), g.reshape(-1,32,768), similarity.reshape(-1)
            optimizer.zero_grad()

            recon_f, mu_f, log_var_f = vae(f)
            recon_g, mu_g, log_var_g = vae(g)

            loss_vae_f = vae_loss(recon_f, f, mu_f, log_var_f)
            loss_vae_g = vae_loss(recon_g, g, mu_g, log_var_g)

            similarity_score = siamese(vae.reparameterize(mu_f, log_var_f), vae.reparameterize(mu_g, log_var_g))
            loss_siamese = F.binary_cross_entropy(similarity_score, similarity.view(-1))

            total_loss = loss_vae_f + loss_vae_g + loss_siamese
            total_loss.backward()
            optimizer.step()
            total_train_loss += total_loss.item()

        average_train_loss = total_train_loss / len(train_dataloader)
        average_val_loss, accuracy, roc_auc, mean_vae_time, mean_seamese_time = validate(vae, siamese, val_dataloader)

        print(f'Epoch {epoch+1}, Train Loss: {average_train_loss}, Validation Loss: {average_val_loss}, Val ROCAUC: {roc_auc}, Time VAE|Seamese: {mean_vae_time}|{mean_seamese_time}')
    return vae, siamese

In [10]:

# Assuming IgorDataset can split between train and validation data
# train_dataset = IgorDataset(train=True)
# val_dataset = IgorDataset(train=False)

# train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Create the model
model_vae = VAE()
model_siamese = SiameseNetwork()

params = list(model_vae.parameters()) + list(model_siamese.parameters())
optimizer = torch.optim.Adam(params, lr=0.01)

# Train the model
vae, siamese = train(model_vae, model_siamese, train_dataloader, test_dataloader, optimizer, epochs=5)

Epoch 1, Train Loss: -7426586.918918919, Validation Loss: -10238333.628378378, Val ROCAUC: 0.5320585247643965, Time VAE|Seamese: 0.00021082621808880933|2.5847753151630773e-06
Epoch 2, Train Loss: -13072428.918918919, Validation Loss: -15704875.121621622, Val ROCAUC: 0.605960830442789, Time VAE|Seamese: 0.00021266699572653727|2.631415719895616e-06
Epoch 3, Train Loss: -18660156.31081081, Validation Loss: -20463410.58108108, Val ROCAUC: 0.6268601133054299, Time VAE|Seamese: 0.0002105139174580075|2.555783873325036e-06
Epoch 4, Train Loss: -22929516.716216218, Validation Loss: -26329881.56756757, Val ROCAUC: 0.6646118608646243, Time VAE|Seamese: 0.00020842076884305643|2.5656030786854233e-06
Epoch 5, Train Loss: -27275004.905405406, Validation Loss: -24232978.41891892, Val ROCAUC: 0.672254655828891, Time VAE|Seamese: 0.00020604619543361727|2.595201860206404e-06


In [11]:
# Epoch 1, Train Loss: 8642097.388888888, Validation Loss: -11821122.333333334, Val ROCAUC: 0.5434709826989619, Time VAE|Seamese: 0.0002096242483924417|2.9477091396556183e-06
# epochs=5 357 min

In [12]:
torch.save(vae.state_dict(), 'vae.pth')
torch.save(siamese.state_dict(), 'siamese.pth')

In [16]:
params2 = list(vae.parameters()) + list(siamese.parameters())
optimizer2 = torch.optim.Adam(params2, lr=0.005)
vae20, siamese20 = train(vae, siamese, train_dataloader, test_dataloader, optimizer2, epochs=15)

Epoch 1, Train Loss: -30918548.62162162, Validation Loss: -34147803.08108108, Val ROCAUC: 0.6746219889940896, Time VAE|Seamese: 0.00021708888822674462|2.706512450961768e-06
Epoch 2, Train Loss: -36733830.94594595, Validation Loss: -40101792.94594595, Val ROCAUC: 0.7177965991546618, Time VAE|Seamese: 0.00021505787210281685|2.7028421495264916e-06
Epoch 3, Train Loss: -43212824.08108108, Validation Loss: -46717901.91891892, Val ROCAUC: 0.7870562471153408, Time VAE|Seamese: 0.00021931634755038983|2.6213076178211817e-06
Epoch 4, Train Loss: -48002734.13513514, Validation Loss: -50523291.567567565, Val ROCAUC: 0.83646279486258, Time VAE|Seamese: 0.000223435794737245|2.6677280671009805e-06
Epoch 5, Train Loss: -52090052.432432435, Validation Loss: -53504693.83783784, Val ROCAUC: 0.8736839990358461, Time VAE|Seamese: 0.0002250155377792456|2.739797948504051e-06
Epoch 6, Train Loss: -54766714.37837838, Validation Loss: -56212497.567567565, Val ROCAUC: 0.9072938604212563, Time VAE|Seamese: 0.0002

In [17]:
torch.save(vae20.state_dict(), 'vae_epoch_20.pth')
torch.save(siamese20.state_dict(), 'siamese_epoch_20.pth')

In [13]:
model_vae = VAE()
model_vae.load_state_dict(torch.load('vae_epoch_20.pth'))
model_siamese = SiameseNetwork()
model_siamese.load_state_dict(torch.load('siamese_epoch_20.pth'))

<All keys matched successfully>

In [14]:
df = df_test.copy()
offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])
test_dataset = MyDataset(offer_batch, true_match_batch, false_match_batch, checkpoint)
test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=True)

In [15]:
df

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,external_brand,external_type,true_match,false_match
31099,Apple iPhone 12 Mini 128ГБ Черный,7883271,1795941692,False,мобильные телефоны,2801,33890.0,"[{'id': 9, 'type': 'r', 'value': 162.0}, {'id'...",Apple iPhone 12 Mini 128ГБ Черный // Multi-Tou...,iPhone,NaN,NaN,"Apple iPhone 12 mini 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 mini 128Gb, nano-Sim+eSIM, (PR..."
42042,Смартфон Apple iPhone 15 Pro Max 1TB eSim (Тит...,7883754,1883059376,False,мобильные телефоны,2801,138990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Фирменный Смартфон Apple iPhone 15 Pro Max 1TB...,iPhone 15 Pro Max,Apple,Смартфон,"Apple iPhone 15 Pro Max 1Tb, Dual eSIM, Black ...","Apple iPhone 15 Pro 512Gb, nano-Sim+eSIM, Черный"
11991,Смартфон Apple iPhone 14 Pro Max 128GB Space B...,7883570,1847607198,False,мобильные телефоны,2801,104500.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Тип товара: Смартфон | Модель: Apple iPhone 14...,Все товары :: Электроника :: Телефоны :: Мобил...,Apple,Смартфон,"Apple iPhone 14 Pro Max 128Gb, Dual eSIM, Spac...","Apple iPhone 14 Pro Max 128Gb, Dual eSIM, Silver"
32353,Смартфон Apple IPhone 13 Blue 256GB синий,7883341,1806950940,False,мобильные телефоны,2801,93591.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",NaN,Смартфоны и гаджеты :: Смартфоны,Apple,Смартфон,"Apple iPhone 13 256Gb, nano-Sim+eSIM, Синий","Apple iPhone 13 256Gb, nano-Sim+eSIM, Розовый"
25650,Телефон Apple iPhone 12 256Gb Red (MGJJ3RU/A),7883252,1435667964,True,мобильные телефоны,2801,78780.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Тип: смартфон; Версия ОС на начало продаж: iOS...,Телефоны :: Сотовые телефоны,Apple,Телефон,"Apple iPhone 12 256Gb, nano-Sim+eSIM, (PRODUCT...","Apple iPhone 12 256Gb, Dual nano-SIM, Фиолетовый"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,"Apple iPhone 12 Pro, 512 ГБ, золотой",7883300,1628253466,True,мобильные телефоны,2801,178500.0,"[{'id': 14521, 'type': 'b', 'value': 1.0}, {'i...",NaN,Смартфоны,Apple,NaN,"Apple iPhone 12 Pro 512Gb, nano-Sim+eSIM, Gold","Apple iPhone 12 Pro 512Gb, Dual nano-SIM, Синий"
11284,Apple iPhone 13 Pro Max 512Gb (Sierra Blue) (N...,7883405,1477426246,False,мобильные телефоны,2801,148550.0,"[{'id': 14522, 'type': 'b', 'value': 1.0}, {'i...",Apple iPhone 13 Pro Max 512Gb (Sierra Blue) (N...,iPhone :: iPhone 13 Pro Max,NaN,NaN,"Apple iPhone 13 Pro Max 512Gb, nano-Sim+eSIM, ...","Apple iPhone 13 Pro Max 512Gb, nano-Sim+eSIM, ..."
38158,Смартфон Apple iPhone 15 Pro 512GB Natural Tit...,7883752,1810935835,False,мобильные телефоны,2801,178900.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Диагональ экрана: 6,1? Тип экрана: OLED Super ...",Электроника :: Мобильные телефоны :: Apple,NaN,NaN,"Apple iPhone 15 Pro 512Gb, nano-Sim+eSIM, Титан","Apple iPhone 15 Pro 512Gb, nano-Sim+eSIM, Синий"
860,Apple iPhone 14 Pro Max 256ГБ Silver (Серебрис...,7883596,1539618956,True,мобильные телефоны,2801,133750.0,"[{'id': 14519, 'type': 'b', 'value': 1.0}, {'i...",Apple iPhone 14 Pro Max — ультрамощный смартфо...,Телефоны :: Телефоны Apple iPhone,Apple,NaN,"Apple iPhone 14 Pro Max 256Gb, Dual nano-SIM, ...","Apple iPhone 14 Pro Max 256Gb, Dual eSIM, Фиол..."


In [16]:
average_val_loss, accuracy, roc_auc, mean_vae_time, mean_seamese_time = validate(model_vae, model_siamese, test_dataloader)

In [17]:
print(f"average_val_loss = {average_val_loss}, accuracy = {accuracy}, roc_auc = {roc_auc}, mean_vae_time = {mean_vae_time}, mean_seamese_time = {mean_seamese_time}")

average_val_loss = -62430577.428571425, accuracy = 0.9425717852684145, roc_auc = 0.9860408766507534, mean_vae_time = 0.00023211399267675277, mean_seamese_time = 2.6689077286833384e-06
